# ANUGA Lab

## CE377K - River Mechanics
### Kyle Wright
#### March 29-31, 2022

The goal of this Notebook is to give users an introduction to some of the basic functionality of the **ANUGA** hydrodynamic model. This model solves the 2D shallow-water equations on an **unstructured triangular grid**, and offers some key advantages over other models for modeling the hydrodynamics of Earth systems.

In this tutorial, we will build a simple, small, coarse model of the **Onion Creek Watershed**. Different cell blocks will demonstate how one can build the model mesh, initialize the domain, setup the parameters of the model, and run the simulation. 

The point of this script is not to build the best model we can of the domain, but rather to **demonstrate the workflow** of constructing a model that can be modified or replicated for other (better) models. 

These notebooks are written in Python 3, and have been designed to run in the Google `colaboratory` environment, which provides a Jupyter notebook environment running on a virtual machine on the cloud. Nothing needs to be installed locally, as it is entirely run on Google Drive. All you need is a Google account. All of these scripts are stored on GitHub, which is a platform for sharing open-source codes.

To start interacting with the notebook follow the `Open in Colab` link above. Once you're there, click `File` > `Save a Copy in Drive`

## Setup Environment

The first thing we have to do is set up our environment. In Python, this means downloading all the necessary functions we want to use, including ANUGA. 

Run the following cell to install the dependencies and some extra code for visualising on Colaboratory. To run each code block, you can simply type `shift` + `enter`

Wait until you see the comment *(5) Ready to go* before proceeding to subsequent commands. The install should take less than a minute.

NOTE: Tutorial installation is based on the [2018 CSDMS Anuga Clinic](https://github.com/stoiver/anuga-clinic-2018)

In [ ]:
# 
try:
  import os
  os.chdir('/content')
  !git clone https://github.com/wrightky/ANUGA_OnionCreek_Tutorial.git

  # Now install environment using tool
  !/bin/bash /content/ANUGA_OnionCreek_Tutorial/anuga_tools/install_anuga_colab.sh
 
except:
  pass

# Make anuga-clinic code available

if not 'workbookDir' in globals():
    workbookDir = os.getcwd()

import sys
sys.path.append(os.path.join(workbookDir,"ANUGA_OnionCreek_Tutorial"))

In [ ]:
import numpy as np
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
# from scipy.interpolate import interp1d
import pandas as pd
import os
import datetime
# from osgeo import gdal
import anuga
from anuga.utilities import animate
# from anuga import Inlet_operator
from google.colab import files

%matplotlib inline

## Create ANUGA domain and mesh

In the following cell, we read in our model boundary polygon, and **instantiate the model domain and mesh** from that polygon. The internal mesh engine does almost all of the work for us here, we just need to feed it our **domain size**, desired **mesh resolution**, and any **interior polygons** we might want (if we have any). 

Interior polygons are one of the powerful features of ANUGA. We can have any number of regions inside our domain with their own mesh resolution. Is there an area you really care about and want to model in high-resolution? Easy. Is there an area you need to include but don't care a lot about, and can make very coarse to save computational time? Also easy. The only constraint is that the polygons defining the boundary those regions **can't overlap**.

For visualization, we will be making use of ANUGA's `animate.Domain_plotter` tool, which we initialize here and call repeatedly later.

In [ ]:
# ---------------Define Domain Boundary-----------------
bounding_polygon = [[622338.0, 3339415.0], [623005.0, 3341128.0],
                    [626983.0, 3340039.0], [626675.0, 3338530.0]]

# Assign a name to the different kinds of boundaries, needed for BCs later
boundary_tags={'sides': [0,1,3], 'downstream': [2]} # ID of segments, not points

# ---------------Define Geo Reference-------------------
geo_reference = anuga.Geo_reference(zone=14,
                                    datum='wgs84',
                                    projection='UTM',
                                    false_easting=500000,
                                    false_northing=0)

# Mesh Resolution (max area of triangles in each given region)
base_res = 100. # Areas we somewhat care about

# ---------------Generate domain and mesh---------------
domain = anuga.create_domain_from_regions(bounding_polygon, boundary_tags,
                                          maximum_triangle_area=base_res,
                                          mesh_geo_reference=geo_reference,
                                          mesh_filename = 'onion.msh')
# Comment out the interior region command to see what the mesh
# looks like without these features
domain.set_name('OnionCreek')
domain.set_flow_algorithm('DE1')
domain.set_minimum_allowed_height(0.02)  # Only store heights > 2 cm

# Plot mesh
fig = plt.figure(figsize=(5, 5), dpi=250, facecolor='w', edgecolor='k')
dplotter = animate.Domain_plotter(domain)
plt.triplot(dplotter.triang, linewidth=0.1);
plt.axis('scaled')

# Print out some descriptive statistics of our mesh cells
print domain.statistics()

## Create bathymetry/topography

Here we **populate** our new mesh with **topographic information** from an ASCII file. We started by converting from a GeoTIFF to an ASCII file (example command shown below using `gdal_translate`, here I just give you the ASCII directly). Then we do some additional conversions to get this into a filetype ANUGA can use (`.dem`, `.pts`). Finally, we use the resulting `.pts` file to create our model bathymetry.

These steps can be **quite slow** for fine meshes, so if the mesh isn't changing between runs, we recommend **saving the resulting topography** from the domain as a `.csv` and just re-loading that file for future runs. This file will be a long list of elevation values sorted by triangle ID.

*Note: The ASCII file needs to have the same headers as the file used here, which sometimes isn't the default. To repeat these steps, check the ASCII file with a text editor and make sure it has the same headers as the example ASCII files in th ANUGA examples folder. It also needs the accompanying `.prj` file!*

In [ ]:
# If starting from a GeoTIFF, translate to ASCII:
# !gdal_translate -of AAIGrid bathy.tif bathy.asc
# ASCII to DEM conversion
anuga.asc2dem('EG_OnionCreek_2m_EPSG_32614_m.asc', use_cache=False, verbose=True)
# DEM to "points"
anuga.dem2pts('EG_OnionCreek_2m_EPSG_32614_m.dem', use_cache=False, verbose=True)
# Use "points" to populate mesh cells
domain.set_quantity('elevation', 
                    filename='EG_OnionCreek_2m_EPSG_32614_m.pts',
                    use_cache=False,
                    verbose=True,
                    alpha=0.1)

# Plot topography
fig=plt.figure(figsize=(5, 5), dpi= 250, facecolor='w', edgecolor='k')
plt.tripcolor(dplotter.triang, facecolors = dplotter.elev, cmap = 'terrain')
plt.colorbar(fraction=0.04);
plt.title("Elevation");
plt.axis('scaled')

# To save time next time, we can save the result
# topo = domain.quantities['elevation'].centroid_values
# np.savetxt('EG_OnionCreek_2m_EPSG_32614_m.csv', topo, delimiter=",")

## Add boundary conditions and tides

Here we create an artificial tidal signal (based on data from an existing NOAA gauge) and feed our **tide function** into the **boundary condition**. We could just as easily have loaded in real tides and fed ANUGA an intepolation function for that data, but here this will do.

For this, we make use of ANUGA's built-in `Time_boundary` boundary condition, for which we specify [`stage`, `xmomentum`, `ymomentum`] as a function of time (only `stage` is set to vary here). All the other boundaries are designated as a no-flux `Reflective_boundary`. Other BCs we could have used include the constant `Dirichlet_boundary`, or some variation of a `Transmissive_boundary`, the latter of which only works in supercritical flows. (Our attempts to use `Transmissive_boundary` in sub-critical coastal flows have all **blown up**. Not fun!)

In [ ]:
# Measured water levels
tides = pd.read_csv(filename, header=0, names=['datetime', 'WL'])
# Convert date string to datetime, create new column epoch for int numeric time
tides['datetime'] = pd.to_datetime(tides['datetime'])
tides['epoch'] = (tides['datetime'] - tides['datetime'][0]) // pd.Timedelta("1s")
fBC_tides = interp1d(tides['epoch'], tides['WL'], kind='linear')

# Downstream boundary
Bout = anuga.Dirichlet_boundary(domain, function=lambda t: [fBC_tides(t), 0.0, 0.0])
# All other boundaries
Br = anuga.Reflective_boundary(domain)

# Assign BCs to the boundaries we named earlier
domain.set_boundary({'bay': Bout, 'sides': Br})

# Visualize tides
tt = np.linspace(0., 86400., 1000)
plt.figure(figsize=(5,3), dpi=150)
plt.plot(tt, fBC_tides(tt), 'k-')
plt.xlabel('Time [sec]')
plt.ylabel('Stage of BC [m]')

## Add discharge hydrograph

Here we define our **discharge**, and create an **inlet** through which that discharge is added to the model.

The way ANUGA handles discharge is **somewhat odd** compared to other models. Most models define a mass inflow as a boundary condition, and every time-step mass is added appropriately. Due to the way ANUGA's numerical solver works, the boundaries aren't quite ***rigid*** enough to ensure that the proper amount of mass enters the domain this way. Instead, it's recommended to use the built-in `inlet_operator` to add the right amount of mass to the model every time-step, and to make the surrounding boundaries **reflective**. Water will therefore enter our domain from any grid cells that intersect our inlet, somewhat like a flooding bath drain. (Note that **outlets** can be created the same way by specifying a **negative discharge**)

To make a static inlet with a steady discharge value, you add the inlet once and be done with it. Because we want our discharge to vary, we have to update the discharge later inside the loop when we run the model. We do this by modifying the `Q` attribute via `wlo_inlet.Q = new_value`.

In [ ]:
# Unsteady discharge parameters
Q_base = 1000.    # starting and ending flow rate
Q_peak = 3500.
t_peak = 9000.   # (2.5 hr, total sim length is 6 hr)
simtime = 21600.

# ------------Define a hydrograph function-------------
def hydrograph(t, tf, t_peak, Q_peak, Q_base):
    if t <= t_peak:
        # Linear rising limb
        Q = Q_base + (Q_peak - Q_base)/t_peak*t
    else:
        # Linear falling limb
        Q = Q_peak - (Q_peak - Q_base)/(tf - t_peak)*(t - t_peak)
    return Q

# Setup inlets
wlo_inlet = [[652800.0, 3272400.0],[653000.0,3272400.0]] # Coordinates of line
# If this Q were static, we could add this inlet once and be done, e.g.
wlo_inflow = Inlet_operator(domain, wlo_inlet, Q = Q_base)

# Visualize time-varying Q
tt = np.linspace(0., simtime, 1000)
Qplt = [hydrograph(ti, simtime, t_peak, Q_peak, Q_base) for ti in tt]
plt.figure(figsize=(5,3), dpi=150)
plt.plot(tt, Qplt, 'k-')
plt.xlabel('Time [sec]')
plt.ylabel('Discharge in the inlet [$m^3/s$]')

## Add rainfall

Rainfall is added to the model domain at a defined rate (mm/s) over a specified period of time. For this simulation, we will add rainfall at **0.01 mm/s** for a duration of **2 hours**. The rainfall scheme specifed here must align with the simulation time to be defined later. We anticipate running a **6 hr simulation**, so let's say here that we will begin applying this rainfall to the domain after **1 hour** of simulation time. This will dump **7.2 cm (2.83 in.)** on the entire domain. This section is pretty simple -- we are only defining our rainfall intensity and timing. The real action occurs in the model simulation loop below.

*Note: There are many ways to define rainfall (and discharge) in ANUGA. If we were using a real storm event, we would probably have a `.csv` file with time-series data that we could read here, create an interpolation function similar to the one we established for wind, and feed in that function instead of this artificial one.*

In [ ]:
#---------------Define rainfall forcing-------------------
# This will dump 7.2 cm (2.83 in.) on the entire domain
rain_rate = 0.01  # mm/s
rain_leng = 3600*2.  # two hours
rain_start = 3600. # start applying rainfall after one hour

# Define rain as a function of time
def rainfall(t):
    if rain_start <= t < rain_leng + rain_start:
        rate = rain_rate # Constant over window
    else:
        rate = 0.
    return rate

# Append to forcing terms
R = anuga.Rainfall(domain, rate=rainfall)
domain.forcing_terms.append(R)

# Visualize time-varying R
tt = np.linspace(0., simtime, 1000)
RR = [rainfall(ti) for ti in tt]
plt.figure(figsize=(5,3), dpi=150)
plt.plot(tt, RR, 'k-')
plt.xlabel('Time [sec]')
plt.ylabel('Rainfall over the domain [$mm/s$]')

## Final touches for initial conditions

Here we initialize the **stage**, as well as Manning's coefficients for the non-vegetated areas of the landscape (channels, the bay). We could have used a dry bed as our initial condition, but this will help the model reach a stable condition more quickly. This run is what we would call a **cold start**, because the model is pretty much starting from scratch with an uninformative initial condition. Ideally, for future runs, we would use the saved output of this run as our new IC, because a **hot start** will always converge more quickly.

In [ ]:
# ---------------Load pre-established elevation----------------------
stage = topo.copy()  # Initialize stage as = topography
stage[topo <= fBC_tides(0)] = fBC_tides(0) # Used for cold start
domain.set_quantity('stage', stage, location='centroids')  # Initialize depth

# Note that if we had initial conditions from a prior run, we could use those:
# domain.set_quantity('xmomentum', init_xmom, location='centroids')
# domain.set_quantity('ymomentum', init_ymom, location='centroids')

# Use Manning's for non-veg regions
friction = np.zeros_like(topo)
friction[topo < -0.6] = 0.02 # Natural channels, nc
friction[topo >= 0.6] = 0.05 # Very rough floodplains (trees)
domain.set_quantity('friction', friction, location = 'centroids')

## Run the model

Here we call on `domain.evolve()` to **run the model** for a specified amount of time. We **save information** on the model's converved quantities (`xmomentum`, `ymomentum`, `stage`) into the output file every `yieldstep`, here set to every **900 seconds** (15 minutes). This **isn't the same thing as the *timestep***: the actual timesteps the model takes are decided by ANUGA based on **stability conditions**, which depends on the spacing of the mesh, speed of the flow, and other numerical mumbo-jumbo. We use the command `domain.print_timestepping_statistics()` to tell us some summary statistics of these timesteps every `yieldstep`.

This loop also enables us to **make changes to our model** as time progresses, if we wish. In this example, we update discharge and wind speed (if turned on) within the evolve loop. To do this, we simply modify the attributes of the discharge and wind objects we created earlier. For each of these functions, we use the value _centered_ in each time window between this yieldstep and the next (i.e. `t + savefreq`). Notice that we do not have to modify the tide or rainfall forcings here -- those were defined using functions of time, so they are automatically updated every time-step. 

In [ ]:
# ------------------------------------------------------------------------------
# Evolve system through time
# ------------------------------------------------------------------------------
savefreq = 900.
simtime = 21600.

for n, t in enumerate(domain.evolve(yieldstep=savefreq, finaltime=simtime)):
    # Print out some statistics on the model evolution
    domain.print_timestepping_statistics()

    # Use animate to save some images to make a movie later
    dplotter.save_stage_frame(figsize=(5, 5), dpi=200, vmin=0, vmax=1)
    
    # -------------Define discharge for current yield step---------------
    wlo_inflow.Q = hydrograph(t+savefreq, simtime, t_peak, Q_peak, Q_base)

    # ---------------Update wind for next yield step----------------
    if wind_on:
        W.speed = fBC_windspeed(t+savefreq/2)
        W.phi = fBC_winddir(t+savefreq/2)  

## Inspect outputs

Here, we introduce a few ways one can query the resulting output file. All of ANUGA's results are saved in a **NetCDF** file with the extension `.sww`. These can be loaded using any normal NetCDF reader, but ANUGA also comes pre-packaged with several functions for inspecting your model results, most of which are stored in `anuga.utilities.plot_utils`

In [ ]:
# Read in the png files stored during the evolve loop
dplotter.make_stage_animation() 

In [ ]:
# Load in values of conserved quantities at every cell centroid
swwvals = anuga.utilities.plot_utils.get_centroids('WLD.sww', timeSlices='all')
# Query values: time, x, y, stage, elev, height, xmom, ymom, xvel, yvel, friction, vel, etc
model_starttime = pd.to_datetime('2016-10-15 00:00:00')

# Separate variables just for accessibility
time = swwvals.time
topo = swwvals.elev
x = swwvals.x
y = swwvals.y
depth = swwvals.height
stage = swwvals.stage
u = swwvals.xvel
v = swwvals.yvel
w = np.sqrt(u**2 + v**2)

In [ ]:
# ------------------------------------------------------------------------------
# Plot a single map of velocity at centroids
# ------------------------------------------------------------------------------
# Time index to plot:
time_idx = 12

timeUTC = pd.to_timedelta(time[time_idx], 's') + model_starttime
fig = plt.figure(figsize=(4, 4), dpi=250, facecolor='w', edgecolor='k')
plt.tripcolor(dplotter.triang, facecolors=w[time_idx,:], 
              vmax=0.6, vmin=0, cmap='plasma')
cbar = plt.colorbar(fraction=0.045);
cbar.set_label('Velocity [m]', rotation=270)
plt.title('Velocity : ' + timeUTC.strftime("%b %d %Y, %H:%M"));
plt.axis('scaled')

In [ ]:
# ------------------------------------------------------------------------------
# Plot a single map of stage at centroids
# ------------------------------------------------------------------------------
# Time index to plot:
time_idx = 12

timeUTC = pd.to_timedelta(time[time_idx], 's') + model_starttime
fig = plt.figure(figsize=(4, 4), dpi=250, facecolor='w', edgecolor='k')
plt.tripcolor(dplotter.triang, facecolors=stage[time_idx,:], 
              vmax=0.8, vmin=0, cmap='jet')
cbar = plt.colorbar(fraction=0.045);
cbar.set_label('Stage [m]', rotation=270)
plt.title('Stage : ' + timeUTC.strftime("%b %d %Y, %H:%M"));
plt.axis('scaled')

In [ ]:
# ------------------------------------------------------------------------------
# Plot time-series at a gauge location
# ------------------------------------------------------------------------------
# Create triangle lookup function to find cell index of a gauge
p = anuga.utilities.plot_utils.get_output('WLD.sww')
tri_lookup = anuga.utilities.plot_utils.get_triangle_lookup_function(p)

# Create time vector of model outputs
model_timeseries = model_starttime + pd.to_timedelta(time, 's')

# Find coordinates of gauge and cell containing it, extract stage
xx, yy = [650620.941, 3265354.835]
idx = tri_lookup(xx, yy)
gaugestage = stage[:, idx]

fig = plt.figure(figsize=(5, 3), dpi=150, facecolor='w', edgecolor='k')
plt.plot(model_timeseries, gaugestage, 'k--')
plt.ylabel('Stage [m NAVD88]')